In [ ]:
import numpy as np
from scipy.io import mminfo,mmread
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.linalg import eig
from numpy import sqrt, dot, sum, abs, diag, array, pi
from scipy import sparse

from numpy.fft import rfft, rfftfreq

Exercise 5 - Identification Techniques
===============================

This exercise is focused on identifying modal parameters from experiments.

Measurement data from a numerical impact hammer experiment on the plate supported by flexible springs is provided
in the file `measurement.txt`.
Time histories of acceleration at 24 points and of the force at the impact point were measured.
The point-coordinates are given in `measurement-coordinates.txt`.

You can load the data with
```python
from numpy import loadtxt
data = loadtxt('measurement.txt')
```

In [ ]:
from numpy import loadtxt
data = loadtxt('measurement.txt')

* Plot the time history for the force as well as its frequency spectrum.

In [ ]:
t = data[:,0] #Time vector
F = data[:,1] #Force vector

In [ ]:
plt.figure(figsize = [15,5])
plt.plot(t,F,label = 'F(t)')
plt.xlabel('t, in s')
plt.ylabel('F, in N')
plt.title('Applied Force')
plt.xlim([0,0.1])
plt.legend()
plt.grid()

In [ ]:
frequencies = rfftfreq(len(t),t[1] - t[0])
F_rfft = rfft(F)

* Plot the measured acceleration time histories and frequency spectra for the 4 corner points and the drive point (where the force is applied). Make sure to select appropriate time and frequency limits to only show useful data.

In [ ]:
plt.figure(figsize = [15,10])
plt.xlabel('t, in s')
plt.ylabel('a, in m/s^2')
plt.title('Acceleration time histories')
plt.xlim([0,1])

for i in range(2,26) :
    plt.plot(t,data[:,i],label = 'a' + str(i-1))
    
plt.legend(loc = 1)
plt.grid()

Now we calculate the frequeny spectra of the 4 corner points. Form the Numerical Experiment Jupyter Notebook we acquire following information:
 * **P01** - Left lower corner
 * **P06** - Right lower corner
 * **P19** - Left top corner
 * **P24** - Right top corner
 
For the injection point of the force: **P05**

In [ ]:
PO1_rfft = rfft(data[:,2])
PO6_rfft = rfft(data[:,7])
P19_rfft = rfft(data[:,20])
P24_rfft = rfft(data[:,25])

P05_rfft = rfft(data[:,6])

Now let's plot the frequency spectra

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(PO1_rfft), label='PO1_rfft')
plt.semilogy(frequencies,np.abs(PO6_rfft), label='PO6_rfft')
plt.semilogy(frequencies,np.abs(P19_rfft), label='P19_rfft')
plt.semilogy(frequencies,np.abs(P24_rfft), label='P24_rfft')
plt.semilogy(frequencies,np.abs(P05_rfft), label='P05_rfft')

plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Frequency Spectra')
plt.legend()
plt.grid()

* Compute the accelerance and plot it for the drive point. 

Make sure to apply a suitable window to the data, to counteract the effect of poor signal to noise ratio in parts of the singal.
For decaying signal a decaring exponential window is suitable, e.g. use
```python
from scipy.signal import windows
frac = 1e-3 # choose something useful
W = windows.exponential(len(t), 0, -(len(t)-1) / np.log(frac), False)
```
Multiply the window to the data, before you do the FFT.


In [ ]:
from scipy.signal import windows
frac = 5e-3 # choose something useful
W = windows.exponential(len(t), 0, -(len(t)-1) / np.log(frac), False)

Let's first compare for one point how the window function influences the acceleration

In [ ]:
plt.figure(figsize = [15,5])
plt.plot(t,data[:,6]*W, label = "P01_window")
plt.plot(t,data[:,6], label = "P01")
plt.xlabel('t, in s')
plt.ylabel('P01, in m/sec^2')
plt.xlim([0,1])
plt.legend()
plt.grid()

In [ ]:
PO1_rfft_window = rfft(data[:,2]*W)
PO6_rfft_window = rfft(data[:,7]*W)
P19_rfft_window = rfft(data[:,20]*W)
P24_rfft_window = rfft(data[:,25]*W)

P05_rfft_window = rfft(data[:,6]*W)

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(PO1_rfft_window), label='PO1_rfft_window')
plt.semilogy(frequencies,np.abs(PO6_rfft_window), label='PO6_rfft_window')
plt.semilogy(frequencies,np.abs(P19_rfft_window), label='P19_rfft_window')
plt.semilogy(frequencies,np.abs(P24_rfft_window), label='P24_rfft_window')
plt.semilogy(frequencies,np.abs(P05_rfft_window), label='P05_rfft_window')

plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Frequency Spectra')
plt.legend()
plt.grid()

* Compute the receptance from the accelerance and plot it for the drive point.

In [ ]:
P05_receptance = rfft(data[:,6]*W)/rfft(data[:,1]*W)

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(P05_receptance))

plt.ylabel('a/F [m/s^2/N]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Receptance from P05/F')
plt.grid()

## Task1: Find Peaks in the Transfer Functions
Use the Maximum Amplitude, Maximum Quadrature or Maximum Quadrature Component method to find all natural frequencies up to 150Hz.

For low frequency peaks it might me best to look at receptance curves, wheras for high frequenies the peaks in the accelerance might be more pronounced.
Some automatic peak finding algorithms are implemented in scipy. 
A useful and rather robust automatic peak-detection tool is the continuous wavelet transform, e.g. use
```python
from scipy.signal import find_peaks, find_peaks_cwt
```
Often the automatic selection has to be fine tuned by hand.

* Use an automatic tool to detect peaks in the drive-point accelerance, and plot the found peaks.
* Then adapt the result by hand (add shift peaks) until you have found all natural frequencies.
* Plot the magnitude, real and imaginary part of the fransfer functions around each peak in a suitable frequency range (~10Hz) to verify the peak location: all transfer functions in one plot, one plot per part, one plot per peak

In [ ]:
from scipy.signal import find_peaks, find_peaks_cwt

In [ ]:
# Use an automatic tool to detect peaks in the drive-point accelerance
peaks_idx_P05_rfft_window, _ = find_peaks(np.abs(P05_rfft_window), height=2000) # The height is set so that only realistic peaks are taken into account

In [ ]:
plt.figure(figsize = [15,7])
plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(-1,151)
plt.title('Found peaks from P05 accelerance')
plt.grid()
plt.plot(frequencies, np.abs(P05_rfft_window))
plt.plot(frequencies[peaks_idx_P05_rfft_window], np.abs(P05_rfft_window)[peaks_idx_P05_rfft_window], 'x')
plt.show()

In [ ]:
# Plot the magnitude
for i in range(0, len(peaks_idx_P05_rfft_window)):
    
    plt.figure(figsize = [15,7])
    plt.semilogy(frequencies/frequencies[peaks_idx_P05_rfft_window[i]], np.abs(P05_receptance))
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.xlim((frequencies[peaks_idx_P05_rfft_window[i]]-10)/frequencies[peaks_idx_P05_rfft_window[i]], (frequencies[peaks_idx_P05_rfft_window[i]]+10)/frequencies[peaks_idx_P05_rfft_window[i]])
    plt.plot(frequencies[peaks_idx_P05_rfft_window]/frequencies[peaks_idx_P05_rfft_window[i]], np.abs(P05_receptance)[peaks_idx_P05_rfft_window], 'x')
    plt.title('Point'+str(i+1))
    plt.grid()

In [ ]:
# Plot the real part
for i in range(0, len(peaks_idx_P05_rfft_window)):
    
    plt.figure(figsize = [15,7])
    plt.semilogy(frequencies/frequencies[peaks_idx_P05_rfft_window[i]], np.real(P05_receptance))
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.xlim((frequencies[peaks_idx_P05_rfft_window[i]]-10)/frequencies[peaks_idx_P05_rfft_window[i]], (frequencies[peaks_idx_P05_rfft_window[i]]+10)/frequencies[peaks_idx_P05_rfft_window[i]])
    plt.plot(frequencies[peaks_idx_P05_rfft_window]/frequencies[peaks_idx_P05_rfft_window[i]], np.real(P05_receptance)[peaks_idx_P05_rfft_window], 'x')
    plt.title('Point'+str(i+1))
    plt.grid()

In [ ]:
# Plot the imaginary part
for i in range(0, len(peaks_idx_P05_rfft_window)):
    
    plt.figure(figsize = [15,7])
    plt.semilogy(frequencies/frequencies[peaks_idx_P05_rfft_window[i]], np.imag(P05_receptance))
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.xlim((frequencies[peaks_idx_P05_rfft_window[i]]-10)/frequencies[peaks_idx_P05_rfft_window[i]], (frequencies[peaks_idx_P05_rfft_window[i]]+10)/frequencies[peaks_idx_P05_rfft_window[i]])
    plt.plot(frequencies[peaks_idx_P05_rfft_window]/frequencies[peaks_idx_P05_rfft_window[i]], np.imag(P05_receptance)[peaks_idx_P05_rfft_window], 'x')
    plt.title('Point'+str(i+1))
    plt.grid()

In [ ]:
frequencies[peaks_idx_P05_rfft_window]

## Determine Damping Ratio
Determine the damping ratio of each natural frequency from the half power width.
Use the drive point receptance.

* plot the determined half power points on the transfer function for each peak

In [ ]:
half_power = np.abs(P05_receptance)/sqrt(2)
half_power[peaks_idx_P05_rfft_window]

In [ ]:
# find out to which index the half power points belong
index_half_power = np.zeros([len(peaks_idx_P05_rfft_window), 4])

for n in range(len(peaks_idx_P05_rfft_window)):
    index_natural_frequencies = peaks_idx_P05_rfft_window[n]

    for i in range(index_natural_frequencies, 0, -1):
        if half_power[index_natural_frequencies] >= np.abs(P05_receptance)[i]:
            index_half_power[n,0] = i
            index_half_power[n,1] = i+1
            break

    for i in range(index_natural_frequencies, len(frequencies)+1, 1):
        if half_power[index_natural_frequencies] >= np.abs(P05_receptance)[i]:
            index_half_power[n,3] = i
            index_half_power[n,2] = i-1
            break

# linear interpolation between between the values found
freq_half_power = np.zeros([len(peaks_idx_P05_rfft_window),2])
    
for n in range(len(peaks_idx_P05_rfft_window)):
    
        freq_half_power[n,0] = ((frequencies[int(index_half_power[n,1])]-frequencies[int(index_half_power[n,0])])/(np.abs(P05_receptance)[int(index_half_power[n,1])]-np.abs(P05_receptance)[int(index_half_power[n,0])]))*(half_power[peaks_idx_P05_rfft_window[n]]-np.abs(P05_receptance)[int(index_half_power[n,0])])+frequencies[int(index_half_power[n,0])]
        freq_half_power[n,1] = ((frequencies[int(index_half_power[n,3])]-frequencies[int(index_half_power[n,2])])/(np.abs(P05_receptance)[int(index_half_power[n,3])]-np.abs(P05_receptance)[int(index_half_power[n,2])]))*(half_power[peaks_idx_P05_rfft_window[n]]-np.abs(P05_receptance)[int(index_half_power[n,2])])+frequencies[int(index_half_power[n,2])]



In [ ]:
# Plot the magnitude
for i in range(0, len(peaks_idx_P05_rfft_window)):
    plt.figure(figsize = [15,7])
    plt.semilogy(frequencies/frequencies[peaks_idx_P05_rfft_window[i]], np.abs(P05_receptance))
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.xlim((frequencies[peaks_idx_P05_rfft_window[i]]-10)/frequencies[peaks_idx_P05_rfft_window[i]], (frequencies[peaks_idx_P05_rfft_window[i]]+10)/frequencies[peaks_idx_P05_rfft_window[i]])
    plt.plot(frequencies[peaks_idx_P05_rfft_window]/frequencies[peaks_idx_P05_rfft_window[i]], np.abs(P05_receptance)[peaks_idx_P05_rfft_window], 'x')
    plt.plot(freq_half_power[i,0]/frequencies[peaks_idx_P05_rfft_window[i]], half_power[peaks_idx_P05_rfft_window[i]], 'o')
    plt.plot(freq_half_power[i,1]/frequencies[peaks_idx_P05_rfft_window[i]], half_power[peaks_idx_P05_rfft_window[i]], 'o')
    plt.title('Point'+str(i+1))
    plt.grid()

In [ ]:
freq_half_power

In [ ]:
frequencies[peaks_idx_P05_rfft_window]

* what damping factors do you obtain?

In [ ]:
# Damping factor
Damping_factor = (freq_half_power[:,1]-freq_half_power[:,0])/(2*frequencies[peaks_idx_P05_rfft_window[:]])

In [ ]:
Damping_factor

## Circle Fitting
Employ the circle fitting algorithm to determine natural frequencies and damping factors.

* Develop a function that fits a circle in the nyquist plane to given data points (receptance values and corresponding frequencies). It should return the natural frequency, damping factor and complex valued modal constant.

* First test your function on a nice, prominant peak of the drive point receptance (e.g. around 73Hz): Select suitable data points (frequency range) for the fit. Plot your fit in the Nyquist plane. Also plot the transfer function estimation by the cicle fit, i.e. $$ H(\omega) = \frac{C_k}{\omega_k^2 - \omega^2 + 2j \omega_k\zeta_k\omega}+D_k$$ in a wider frequency range, and compare it to the data to see how the fit performs.

* Use your function and determine suitable frequency ranges to fit all natural frequencies. For some it might be useful to switch to a different transfer function, where the peak under consideration is more prominant.

* Use the circle fit algorithm to determine the modal constants for all measured locations (=recpetance functions). Use the frequency ranges (around the peaks) determined above.

* Do the natural frequecies and damping factors obtained from different transfer functions differ?

* Use the moda parameters obtained via the circle fit to estimate the drive point recepance. Plot the measured repeptance, the single dregree of freedom estimates from the circle fit, as well as their sum (all in one plot to compare).

## Task 2: Compute the Mode Shapes

Having obtained natural frequencies and damping factors, from peak-picking (maximum maplitude, quadrature component, ...) use the recepance curves to determine the modal constants and mode shapes.
The mode shapes will be complex valued in ganeral (due to the damping present in the system). As the are complex-valued you can display them as real/imaginary part, or absolute value and phase. Try both!

In [ ]:
def plotmode2d(v,x,y,lim=None,ax=None) :
    """plots a mode v defined at the points (x,y)"""
    if lim==None:
        lim = np.max(np.abs(v))
    if ax==None:
        fig,ax = plt.subplots(figsize=[3.5,2])
        returns = (fig,ax)
    else :
        returns = ax
    ax.tricontourf(x,y,v,cmap=plt.get_cmap('RdBu_r'),vmin=-lim,vmax=lim)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    #fig.tight_layout()
    return returns

coordinates = loadtxt('measurement-coordinates.txt')

* Compute and plot the mode shapes from the peak-picking procedure

In [ ]:
#Compute rffts for every point
P_rfft_window = np.zeros((3043,24),dtype= complex)
H_aa = np.zeros((3043,24),dtype= complex)
for i,j in enumerate(range(2,26),0):
    P_rfft_window[:,i] = rfft(data[:,j]*W)
    H_aa[:,i] = rfft(data[:,j]*W)/rfft(data[:,j]*W)

In [ ]:
#Compute modal constants
Modal_constants = np.zeros((5,24),dtype= complex)
for i in range(P_rfft_window.shape[1]):
    Modal_constants[:,i]= P_rfft_window[:,i][peaks_idx_P05_rfft_window]*2j*frequencies[peaks_idx_P05_rfft_window]**2*Damping_factor

In [ ]:
#Compute modes
Input_modes = np.zeros((5,24), dtype =complex)
Output_modes = np.zeros((5,24), dtype =complex)
for i in range(H_aa.shape[1]):
    Input_modes[:,i] = (2j*frequencies[peaks_idx_P05_rfft_window]**2*Damping_factor*H_aa[:,i][peaks_idx_P05_rfft_window])**0.5
    Output_modes[:,i] = Modal_constants[:,i]/Input_modes[:,i]

* Compute and plot the modes from circle fitting

* animate selected mode shapes. What do you observe?

## Ibrahim Time Domain Method

Implement the Ibrahim Time Domain method to use it on the data from the impact hammer experiemnt.
Since the ITD operates on the free oscillation response in terms of the degrees of freedom (displacements in our case) you need to either compute the impulse respone from the accelerance using an inverse FFT or use the provided displacement data in `measurement-ITD.txt`.

* implement the ITD alogithm. Use the following skeleton

```python
def ITD(D,dt=1.0,n1=1,n3=17,tol=1.0):
    """Ibrahim time domain identification alogithm
    
    Parameters
    ----------
    D : array(N,M)
        matrix containing data at N measurement positions
        time sampled at M times
    dt : float
        sampling interval
    n1 : int
        number samples to shift between measurement matrix X1 and
        its time shifted companion X2
    n3 : int
        number samples to shift between upper and lower part of
        X1 and X2, respectively
    tol : float
        tolerance for the MSCCF used to sort out computational modes.
        only modes abs(MSCCF-1)<tol are returned. 
        
    Returns
    -------
    lam : array(K)
        identified eigenvalues, only positive frequencies are returned
    V : array(N,K)
        corresponding eigenvectors
    MSCCF : array(K)
        the mode shape correlation and confidence factor
    """
    
    # 1) determine which samples should be selected in X1 and X2
    Nt = D.shape[1]
    It1 = arange(0,Nt-n3-n1,1)
    # 2) select samples from data matrix
    X1u = D[:,It1] # upper part
    X1l = D[:,It1+n3] # lower part (shifted by n3 samples)
    X2u = D[:,It1+n1] # time shifted companion (shifted by n1 samples)
    X2l = D[:,It1+n1+n3]
    # 3) assmble X1 and X2
    # 4) compute A: use a robust inverse like numpy's linalg.pinv, or solve linear systems
    # 5) solve eigenvalue problem
    # 6) convert eigenvalues
    # 7) compute MSCCF and choose what to return
    
    return lam, V
```

* use the ITD to identify modes from the impulse response. Choose what data to take (which time range is useful?), and what sampling parameters are suitable. Plot the input data over time (all signals in one plot).

* plot the obtained modes and give their natural frequencies, damping factors, and MSCCFs.

## Compare mode shapes

Use the MAC / MSF to compare your identified modes with the deformation modes of the free-free plate.

* Compute the free-free modes and interpolate/evaluate them at the measurement locations.

* Compute the MAC-matrix between the "experimental" modes from peak-picking and the "numeric" free-free modes. Plot the matrix and ude it to identify the best-fitting "experimental" mode for each "numeric" deformation mode.

* Plot the mode shapes of the correlated mode pair to check. Do the frequencies match? Which threshold value of the MAC is suitable to select "correlated" modes?

* Do the same for the modes from circle fitting. Mark the impact location in the plots. Give frequencies and damping ratios, as well as absolute value and phase of the MSF between "numeric" and "experimental" modes.

* Do the same for the modes form the ITD